<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/01_pytorch_workflow_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. PyTorch Workflow Exercise Template

The following is a template for the PyTorch workflow exercises.

It's only starter code and it's your job to fill in the blanks.

Because of the flexibility of PyTorch, there may be more than one way to answer the question.

Don't worry about trying to be *right* just try writing code that suffices the question.

You can see one form of [solutions on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/solutions) (but try the exercises below yourself first!).

In [ ]:
# Import necessary libraries
import torch
from torch import nn

In [ ]:
# Setup device-agnostic code
device = 'gpu' if torch.cuda.is_available() else 'cpu'

## 1. Create a straight line dataset using the linear regression formula (`weight * X + bias`).
  * Set `weight=0.3` and `bias=0.9` there should be at least 100 datapoints total. 
  * Split the data into 80% training, 20% testing.
  * Plot the training and testing data so it becomes visual.

Your output of the below cell should look something like:
```
Number of X samples: 100
Number of y samples: 100
First 10 X & y samples:
X: tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900])
y: tensor([0.9000, 0.9030, 0.9060, 0.9090, 0.9120, 0.9150, 0.9180, 0.9210, 0.9240,
        0.9270])
```

Of course the numbers in `X` and `y` may be different but ideally they're created using the linear regression formula.

In [ ]:
# Create the data parameters
weight = 0.3
bias = 0.9

# Make X and y using linear regression feature
X = torch.arange(0, 1, 0.01)
y = weight* X + bias

print(f"Number of X samples: {len(X)}")
print(f"Number of y samples: {len(y)}")
print(f"First 10 X & y samples:\nX: {X[:10]}\ny: {y[:10]}")

In [ ]:
# Split the data into training and testing
split = 0.8
train_size = int(split* X.shape[0])
X_train, X_test = X[0:train_size], X[train_size:]
y_train, y_test = y[0:train_size], y[train_size:]

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"Y_train: {y_train.shape}")
print(f"Y_test: {y_test.shape}")

X

In [ ]:
# Plot the training and testing data 
import matplotlib.pyplot as plt

def plot(X= None, y=None, training_data = False):
    plt.scatter(X_train,y_train, c = 'r', marker = 'o', linewidths = .1)
    plt.scatter(X_test, y_test, c = 'b', marker = 'o', linewidths = .1)
    if training_data:
        plt.scatter(X, y, c = 'y', marker = 'o', linewidths = .1)

plot()

## 2. Build a PyTorch model by subclassing `nn.Module`. 
  * Inside should be a randomly initialized `nn.Parameter()` with `requires_grad=True`, one for `weights` and one for `bias`. 
  * Implement the `forward()` method to compute the linear regression function you used to create the dataset in 1. 
  * Once you've constructed the model, make an instance of it and check its `state_dict()`.
  * **Note:** If you'd like to use `nn.Linear()` instead of `nn.Parameter()` you can.

In [ ]:
# Create PyTorch linear regression model by subclassing nn.Module
class  LinearModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1), requires_grad=True)
        self.bias = nn.Parameter(torch.randn(1), requires_grad=True)

    def forward(self, x):
        return self.weights*x + self.bias

class LinearModel_v2(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features= 1, out_features=1)

    def forward(self, x):
        return self.linear(x)

In [ ]:
# Instantiate the model and put it to the target device
model = LinearModel().to(device)
print(model)
print(list(model.parameters()))

model.to(device)
# Model 2
model_2 = LinearModel_v2().to(device)
print(model_2)
print(list(model_2.parameters()))

model_2.to(device)

## 3. Create a loss function and optimizer using `nn.L1Loss()` and `torch.optim.SGD(params, lr)` respectively. 
  * Set the learning rate of the optimizer to be 0.01 and the parameters to optimize should be the model parameters from the model you created in 2.
  * Write a training loop to perform the appropriate training steps for 300 epochs.
  * The training loop should test the model on the test dataset every 20 epochs.

In [ ]:
# Create the loss function and optimizer
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
# Training loop
torch.manual_seed(42)

# Train model for 300 epochs
epochs = 300


# Send data to target device
X_train = X_train.to(device)
X_test = X_test.to(device)

y_train = y_train.to(device)
y_test = y_test.to(device)

for epoch in range(epochs):
  ### Training
  # Put model in train mode
    model.train()
  

  # 1. Forward pass
    y_train_pred = model(X_train)  

    # 2. Calculate loss
    loss = loss_fn(y_train_pred, y_train)

    # 3. Zero gradients
    optimizer.zero_grad()
  

    # 4. Backpropagation
    loss.backward()

    # 5. Step the optimizer
    optimizer.step()
  

    ### Perform testing every 20 epochs
    if epoch % 20 == 0:

    # Put model in evaluation mode and setup inference context 
        model.eval()

        with torch.inference_mode():
      # 1. Forward pass
            y_test_pred = model(X_test)
      # 2. Calculate test loss
            test_loss = loss_fn(y_test_pred, y_test)

      # Print out what's happening
        print(f"Epoch: {epoch} | Train loss: {loss:.3f} | Test loss: {test_loss:.3f}")

## 4. Make predictions with the trained model on the test data.
  * Visualize these predictions against the original training and testing data (**note:** you may need to make sure the predictions are *not* on the GPU if you want to use non-CUDA-enabled libraries such as matplotlib to plot).

In [ ]:
# Make predictions with the model
model.eval()

with torch.inference_mode():
    y_pred = model(X_test)

print(y_pred.shape)
print(y_pred[:10])

# pring model parameters
list(model.parameters())

In [ ]:
plot(X_test, y_pred, True)

## 5. Save your trained model's `state_dict()` to file.
  * Create a new instance of your model class you made in 2. and load in the `state_dict()` you just saved to it.
  * Perform predictions on your test data with the loaded model and confirm they match the original model predictions from 4.

In [ ]:
from pathlib import Path

# 1. Create models directory 
model_path = Path("model_assignment")
model_path.mkdir(parents=True, exist_ok = True)

# 2. Create model save path 
model_name = "01_pytorch_model_linear"
model_save_path = model_path / model_name

# 3. Save the model state dict
print(f"saving model to {model_save_path}")
torch.save(obj = model.state_dict(), f = model_save_path)

In [ ]:
# Create new instance of model and load saved state dict (make sure to put it on the target device)
loaded_model = LinearModel()
loaded_model.load_state_dict(torch.load(f=model_save_path))
loaded_model.to(device)

In [ ]:
# Make predictions with loaded model and compare them to the previous
test_pred_loaded_model = loaded_model(X_test)

test_pred_loaded_model == y_pred